In [2]:
%pylab inline
import keras
import keras.backend as K
import kapre
import pandas as pd
import scipy.io.wavfile as wav
import numpy as np
import random
import arrow
import threading
import hyperopt
import pprint
import librosa
import samplerate
from hyperopt import hp
from soph import soph_scaler, center_wave, ex_generator

# https://github.com/keunwoochoi/kapre

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.5/dist-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random', 'arrow']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
from lsuv_init import LSUVinit

In [ ]:
delta_kernel = np.arange(-self.n,self.n+1)
delta_kernel

In [6]:
num_classes = 12
num_ex = 32

y_pred = np.random.randint(0,num_classes,num_ex)
y_zo = -np.ones((num_ex,num_classes))
y_zo[np.arange(32),y_pred] = 1

In [7]:
y_zo

array([[-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1.],
       [-1., -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1., -1.],
       [-1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1.],
       [-1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1.,  1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1., -1.],
       [-1., -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1., -1.],
       [-1., -1., -1., -1., -1., -

# test resamplers

In [25]:
src_cute, orig_sr = librosa.load("kapre/srcs/bensound-cute.mp3", sr=None, mono=True)

In [26]:
%%timeit
_ =librosa.core.resample(src_cute,orig_sr=orig_sr, target_sr=8000,res_type='kaiser_best')

391 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%%timeit
_ =librosa.core.resample(src_cute,orig_sr=orig_sr, target_sr=8000,res_type='kaiser_fast')

86 ms ± 151 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
%%timeit
_ =librosa.core.resample(src_cute,orig_sr=orig_sr, target_sr=8000,res_type='scipy')

1.14 s ± 1.88 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%%timeit
_ = samplerate.resample(src_cute, 8000/orig_sr, "sinc_best")

740 ms ± 1.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%%timeit
_ = samplerate.resample(src_cute, 8000/orig_sr, "sinc_medium")

138 ms ± 151 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [31]:
%%timeit
_ = samplerate.resample(src_cute, 8000/orig_sr, "sinc_fastest")

54.6 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
%%timeit
_ = samplerate.resample(src_cute, 8000/orig_sr, "zero_order_hold")

895 µs ± 1.23 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [33]:
%%timeit
_ = samplerate.resample(src_cute, 8000/orig_sr, "linear")

1.14 ms ± 1.08 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [2]:
ex_df = pd.read_pickle("data/ex_df.pkl")
current_time = arrow.now().to('US/Eastern').format('YYYY-MM-DD-HH-mm')

src_space = {
    "dropout_prob": hp.uniform("dropout_prob",0,1), #[0,1]
    "activation": hp.choice("activation", ("elu", "relu")), #{ elu, relu}
    "batch_size": 64, #2^int[1,7]
    "regularize": hp.choice("regularize", (True, False)), #{T,F}
    "l2_reg": hp.loguniform("l2_reg",-10,-1), #log([e-8,e-1])
    "init_stdd": hp.loguniform("init_stdd",-10,-1), #log([e-8,e-1])
    "filters_start": hp.quniform("filters_start",40,200,10), #[40,200]
    "filters_step": hp.quniform("filters_step",0,200,10), #[0,60]
    "kernel_size": hp.quniform("kernel_size",2,10,1), #[1,10]
    "cnn_pad": hp.choice("cnn_pad", ("same", "valid")), #{same,valid}
    "cnn_stride": hp.quniform("cnn_stride",1,5,1), #{1,2}
    "n_mfcc": hp.quniform("n_mfcc",16,100,2), #int[16,100]
    "n_mels": hp.quniform("n_mels",16,200,4), #int[16,200]
    "n_dft": 512, #2^int[8,12]
    "n_hop": 256, #2^int[7,12]
    "pool": hp.choice("pool", ("max", "avg")), #{max,avg}
    "pool_pad": hp.choice("pool_pad", ("same", "valid")), #{same,valid}
    "batch_normalize": hp.choice("batch_normalize", (True, False)), #{T,F}
    "train_state": ["train"],
    "val_state": ["val", "test"],
    "p_transform": hp.uniform("p_transform",0,1), #[0,1]
    "vol_range": hp.uniform("vol_range",0,1), #[0,1]
    "displacement": hp.uniform("displacement",0,1), #[0,1]
}
num_cat = 12

In [3]:
src_args = hyperopt.pyll.stochastic.sample(src_space)
pprint.pprint(src_args)

{'activation': 'elu',
 'batch_normalize': True,
 'batch_size': 64,
 'cnn_pad': 'same',
 'cnn_stride': 3.0,
 'displacement': 0.31403276194871466,
 'dropout_prob': 0.8486683100963217,
 'filters_start': 70.0,
 'filters_step': 140.0,
 'init_stdd': 0.0005752235462514456,
 'kernel_size': 6.0,
 'l2_reg': 0.00011094305866935931,
 'n_dft': 512,
 'n_hop': 256,
 'n_mels': 176.0,
 'n_mfcc': 52.0,
 'p_transform': 0.3941505447463889,
 'pool': 'max',
 'pool_pad': 'valid',
 'regularize': False,
 'train_state': ('train',),
 'val_state': ('val', 'test'),
 'vol_range': 0.8130972187337495}


In [49]:
input_layer = keras.layers.Input(shape=(1, 16000))
input_block = kapre.time_frequency.MFCC(n_mfcc=int(src_args["n_mfcc"]),
                              n_mels=int(src_args["n_mels"]),
                              n_dft=int(src_args["n_dft"]),
                              power_melgram=2.0,
                              return_decibel_melgram=True,)(input_layer)

In [50]:
def pool(x):
    if src_args["pool"] == "max":
        return keras.layers.MaxPool2D(padding=src_args["pool_pad"])(x)
    elif src_args["pool"] == "avg":
        return keras.layers.AvgPool2D(padding=src_args["pool_pad"])(x)

In [51]:
drop = src_args["dropout_prob"]
init = keras.initializers.TruncatedNormal(stddev=src_args["init_stdd"])
if src_args["regularize"]:
    reg = keras.regularizers.l2(src_args["l2_reg"])
else:
    reg = None
    
def cnn(x, i=0):
    return keras.layers.Conv2D(
        filters=int(src_args["filters_start"] + i*src_args["filters_step"]),
        kernel_size=int(src_args["kernel_size"]),
        padding=src_args["cnn_pad"],
        strides=int(src_args["cnn_stride"]),
        activation=src_args["activation"],
        kernel_initializer=init,
        kernel_regularizer=reg,
        bias_regularizer=reg)(x)

In [52]:
def calc_cnn_dim(nin):
    k = src_args["kernel_size"]
    if src_args["cnn_pad"] == "valid":
        p = 0
    else:
        p = k
    s = src_args["cnn_stride"]

    return int(((nin + 2 * p - k) / s) + 1)


def calc_cnn_size(layer, layer_type):
    if layer_type == "cnn":
        k = src_args["kernel_size"]
        if src_args["cnn_pad"] == "valid":
            p = 0
        else:
            p = k
        s = src_args["cnn_stride"]
    elif layer_type == "pool":
        k = 2
        if src_args["pool_pad"] == "valid":
            p = 0
        else:
            p = k
        s = 2
    calc_dim = lambda x: int(((x + 2 * p - k) / s) + 1)
    layer_in = np.array(layer.shape.as_list())
    layer_out = (layer_in[0], 
                 calc_dim(layer_in[1]),
                 calc_dim(layer_in[2]),
                 layer_in[3])
                 
    return np.array(layer_out)[1:]

In [53]:
cnn_block = input_block
continue_flag = True
cnn_i = 0
while continue_flag:
    
    cnn_size_flag = np.all(calc_cnn_size(cnn_block, "cnn") > 0)
    
    cnn_size_flag = cnn_size_flag & (sum(cnn_block.shape.as_list()[1:3]) > 2)
    if cnn_size_flag:
        cnn_block = cnn(cnn_block, cnn_i)
        cnn_i += 1
    else:
        continue_flag = False
        break
    
#     print(calc_cnn_size(cnn_block, "pool"))
    pool_size_flag = np.all(calc_cnn_size(cnn_block, "pool") > 0)
    if pool_size_flag:
        cnn_block = pool(cnn_block)
        
    if src_args["batch_normalize"]:
        cnn_block = keras.layers.BatchNormalization()(cnn_block)
    
    cnn_block = keras.layers.Dropout(src_args["dropout_prob"])(cnn_block)
    

In [54]:
out_block = cnn_block
out_block = keras.layers.Flatten()(out_block)
output_layer = keras.layers.Dense(num_cat, activation="softmax")(out_block)

In [55]:
cnn_model = keras.Model(inputs=input_layer, outputs=output_layer)
cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 1, 16000)          0         
_________________________________________________________________
mfcc_5 (MFCC)                (None, 16, 63, 1)         311216    
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 3, 18, 50)         5050      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 1, 9, 50)          0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 1, 9, 50)          200       
_________________________________________________________________
dropout_15 (Dropout)         (None, 1, 9, 50)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 450)               0         
__________

In [56]:
log_base = "logs/cnn/{}/".format(current_time)
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir=log_base + 'tb',
        batch_size=src_args["batch_size"],
        histogram_freq=0,
        write_grads=False,
        write_images=True
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=log_base + 'model-checkpoint.hdf5',
        monitor='val_loss',
        verbose=1,
        save_best_only=True,
        save_weights_only=False,
        mode='auto',
        period=1),
    keras.callbacks.CSVLogger(log_base + 'training.log'),
    keras.callbacks.EarlyStopping(patience=4, verbose=1),
    keras.callbacks.ReduceLROnPlateau(
        factor=0.5, patience=1, verbose=1, min_lr=1e-8)
]

cnn_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='nadam',
    metrics=['accuracy'])


def launchTensorBoard():
    import os
    os.system('pkill tensorboard')
    os.system('tensorboard --logdir=' + log_base + 'tb')
    return


t = threading.Thread(target=launchTensorBoard, args=([]))
t.start()

val_data = next(ex_generator(
    batch_size=sum(ex_df.state.isin(src_args["val_state"])),
    shuffle=False,
    state=src_args["val_state"],
    vol_range=0,
    displacement=0,
    p_transform=0))

In [57]:
history = cnn_model.fit_generator(
    generator=ex_generator(
        batch_size=src_args["batch_size"],
        shuffle=True,
        state=src_args["train_state"],
        vol_range=src_args["vol_range"],
        displacement=src_args["displacement"],
        p_transform=src_args["p_transform"]),
    steps_per_epoch=sum(ex_df.state.isin(
        src_args["train_state"])) / src_args["batch_size"],
    epochs=200,
    verbose=1,
    max_queue_size=100,
    callbacks=callbacks,
    validation_data=val_data
)

Epoch 1/200
827/826 [==============================] - 23s 28ms/step - loss: 1.3425 - acc: 0.3926 - val_loss: 1.9783 - val_acc: 0.4116
Epoch 2/200
827/826 [==============================] - 21s 25ms/step - loss: 0.8423 - acc: 0.5604 - val_loss: 2.3756 - val_acc: 0.2985
Epoch 3/200
824/826 [============================>.] - ETA: 0s - loss: 0.6620 - acc: 0.6289Epoch 00003: val_loss did not improve

Epoch 00003: reducing learning rate to 0.0010000000474974513.
827/826 [==============================] - 21s 26ms/step - loss: 0.6623 - acc: 0.6286 - val_loss: 2.0609 - val_acc: 0.3424
Epoch 4/200
827/826 [==============================] - 21s 26ms/step - loss: 0.5341 - acc: 0.6818 - val_loss: 1.4344 - val_acc: 0.6202
Epoch 5/200
827/826 [==============================] - 21s 25ms/step - loss: 0.4969 - acc: 0.6961 - val_loss: 1.0873 - val_acc: 0.7198
Epoch 6/200
827/826 [==============================] - 21s 26ms/step - loss: 0.4679 - acc: 0.7111 - val_loss: 1.1660 - val_acc: 0.5330
Epoch 7/20

Epoch 28/200
824/826 [============================>.] - ETA: 0s - loss: 0.2618 - acc: 0.8116Epoch 00028: val_loss did not improve

Epoch 00028: reducing learning rate to 1.9531250927684596e-06.
827/826 [==============================] - 21s 25ms/step - loss: 0.2619 - acc: 0.8114 - val_loss: 0.5054 - val_acc: 0.7827
Epoch 29/200
825/826 [============================>.] - ETA: 0s - loss: 0.2625 - acc: 0.8130Epoch 00029: val_loss did not improve

Epoch 00029: reducing learning rate to 9.765625463842298e-07.
827/826 [==============================] - 21s 25ms/step - loss: 0.2623 - acc: 0.8130 - val_loss: 0.5046 - val_acc: 0.7807
Epoch 00029: early stopping


In [63]:
objective = np.min(history.history["val_loss"])

In [62]:
current_time

'2018-01-04-14-36'

In [66]:
with open("logs/opt-log", "w") as f:
    f.writelines("{}, {}".format(current_time, objective))

In [ ]:
hyperopt.fmin()